In [2]:
import pandas as pd
import numpy as np

In [4]:
train_data = pd.read_csv("res/Titanic_train.csv")
train_data.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,463,0,1,"Gee, Mr. Arthur H",male,47.0,0,0,111320,38.5000,E63,S
1,1,121,0,2,"Hickman, Mr. Stanley George",male,21.0,2,0,S.O.C. 14879,73.5000,NaN,S
2,3,415,1,3,"Sundman, Mr. Johan Julian",male,44.0,0,0,STON/O 2. 3101269,7.9250,NaN,S
3,4,480,1,3,"Hirvonen, Miss. Hildur E",female,2.0,0,1,3101298,12.2875,NaN,S
4,5,622,1,1,"Kimball, Mr. Edwin Nelson Jr",male,42.0,1,0,11753,52.5542,D19,S


In [5]:
train_data['Pclass'][0]

1

In [ ]:
train_data.drop('Unnamed: 0', axis = 1, inplace = True)
#train_data.head()

In [ ]:
sur_3c_df = pd.crosstab(train_data['Pclass'] == 3, train_data['Survived'] == 1).reset_index()
sur_3c_df

In [ ]:
sur_3c = sur_3c_df.iloc[1,2]
sur_3c

In [ ]:
male_died_df = pd.crosstab(train_data['Sex'] == 'male', train_data['Survived'] == 0).reset_index()
male_died_df

In [ ]:
male_died = male_died_df.iloc[1,2]
male_died

In [ ]:
sur_s_df = pd.crosstab(train_data['Embarked'] == 'S', train_data['Survived'] == 1).reset_index()
sur_s_df

In [ ]:
sur_southampton = sur_s_df.iloc[1,2]
sur_southampton

In [ ]:
result = [sur_3c, male_died, sur_southampton]
"""import csv
file = open("output/output.csv", mode = 'w', newline = '')
with file:
    write = csv.writer(file)
    write.writerows(outlist)
file.close()
"""
result = pd.DataFrame(result)
result.to_csv('output/output.csv', header = False, index = False)

In [ ]:
#Question2 & 3
age_missing_values = train_data['Age'].isnull().sum()
age_missing_values

In [ ]:
age_missing_index = 0
index_list = train_data[train_data['Age'].isnull()].index.tolist()
for i in index_list:
    age_missing_index = age_missing_index + i
age_missing_index

In [ ]:
df_test = train_data[train_data['Age'].isnull()]
df_train = train_data[train_data['Age'].notnull()]
df_test.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_test['sex_enc'] = labelencoder.fit_transform(df_test['Sex'])
df_test.head()

In [ ]:
df_train['sex_enc'] = labelencoder.fit_transform(df_train['Sex'])
df_train.head()

In [ ]:
X_train = df_train[['Pclass', 'Survived', 'sex_enc']]
X_train.head()

In [ ]:
y_train = df_train['Age']
y_train.head()

In [ ]:
X_test = df_test[['Pclass', 'Survived', 'sex_enc']]
X_test.head()

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
age_pred = reg.predict(X_test)
age_pred = np.round(age_pred, decimals = 2)
age_pred

In [ ]:
age_mean = int(np.mean(age_pred))
age_mean

In [ ]:
df_test.head()

In [ ]:
df_test.drop(['Age'], axis = 1, inplace = True)
df_test.head()

In [ ]:
df_test['Age'] = age_pred
df_test.head()

In [ ]:
df_con = pd.concat([df_train, df_test], ignore_index = True, sort = False)
df_con.head()

In [ ]:
df_con['AdultOrChild'] = df_con.Age.apply(lambda x: 'Adult' if x >= 18 else 'Child')
df_con.head()

In [ ]:
#age_sur_df = pd.crosstab(df_con['Survived'] == 1, df_con['AdultOrChild']).reset_index()
age_sur_df = pd.crosstab(df_con['Survived'], df_con['AdultOrChild'])
age_sur_df

In [ ]:
age_sur = age_sur_df.iloc[1,0] + age_sur_df.iloc[1,1]
age_sur

In [ ]:
X1_train = df_con[['Pclass', 'Age', 'sex_enc']]
X1_train.head()

In [ ]:
y1_train = df_con['Survived']
y1_train.head()

In [ ]:
test_df = pd.read_csv("res/Titanic_test.csv")
test_df['sex_enc'] = labelencoder.fit_transform(test_df['Sex'])
test_df_age_missing = test_df[test_df['Age'].isnull()]
test_df_age_available = test_df[test_df['Age'].notnull()]
age_test = test_df_age_missing[['Pclass', 'Survived', 'sex_enc']]
test_age_pred = np.round(reg.predict(age_test), decimals = 2)
test_df_age_missing.drop(columns = ['Age'], inplace = True)
test_df_age_missing['Age'] = test_age_pred
test_df = pd.concat([test_df_age_available, test_df_age_missing], ignore_index = True, sort = False)
X1_test = test_df[['Pclass', 'Age', 'sex_enc']]
y1_test = test_df[['Survived']]

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X1_train, y1_train)
y1_pred = log_reg.predict(X1_test)
y1_pred

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
c_mat = confusion_matrix(y1_test, y1_pred)
c_mat

In [107]:
tn, fp, fn, tp = confusion_matrix(y1_test, y1_pred).ravel()
print(tn, tp, fn, fp)

45 40 12 13


In [ ]:
sum_c_mat = tn+tp+fn+fp
sum_c_mat

In [ ]:
acc_c_mat = int((tn+tp)/sum_c_mat*100)
acc_c_mat

In [ ]:
from sklearn.metrics import accuracy_score
acc_log_reg = accuracy_score(y1_test, y1_pred)
acc_log_reg